In [1]:
import requests
import pandas as pd
import ibis
from collections import Counter
from io import StringIO

# Step 1: Download lotto results from the provided URL
def download_lotto_results(url):
    response = requests.get(url)
    if response.status_code == 200:
        csv_data = response.text
        return csv_data
    else:
        print(f"Failed to download data. Status code: {response.status_code}")
        return None

# Step 2: Load lotto results into an Ibis table using Pandas
def load_lotto_results(csv_data):
    # Load CSV data into a Pandas DataFrame, including the Draw date and winning numbers
    csv_file = StringIO(csv_data)
    df = pd.read_csv(csv_file)

    # Convert the 'Draw date' to datetime format
    df['Draw date'] = pd.to_datetime(df['Draw date'], format='%d/%m/%Y')

    # Sort the numbers for each row and format them as a string "num1-num2-num3-num4-num5-num6"
    df['sorted_results'] = df.apply(
        lambda row: '-'.join(map(str, sorted([row['Winning Number 1'], row['Winning Number 2'], row['Winning Number 3'],
                                              row['Winning Number 4'], row['Winning Number 5'], row['Winning Number 6'], row['Winning Number 7']]))),
        axis=1
    )

    # Convert the DataFrame into an Ibis table
    ibis_table = ibis.pandas.connect({'lotto_results': df[['Draw date', 'sorted_results']]}).table('lotto_results')

    return ibis_table

# Step 3: Count occurrences and track the dates for each formatted result using Ibis
def count_combinations_with_dates(ibis_table):
    # Group by the sorted results and count occurrences, also tracking all dates for each combination
    counted = (
        ibis_table.group_by('sorted_results')
        .aggregate(
            count=ibis_table.sorted_results.count(),
            dates=ibis_table['Draw date'].collect()
        )
        .execute()
    )

    # Create a list of tuples for combinations, counts, and concatenated dates
    combination_counts = [
        (row['sorted_results'], row['count'], '|'.join(date.strftime('%d/%m/%Y') for date in row['dates']))
        for _, row in counted.iterrows()
    ]
    
    return combination_counts

# Step 4: Save the results to CSV, sort by Count, and include concatenated dates
def save_counts_to_csv(combination_counts, output_filename):
    # Convert the combination counts to a Pandas DataFrame
    df_results = pd.DataFrame(combination_counts, columns=['Combination', 'Count', 'Dates'])

    # Sort by Count in descending order
    df_results = df_results.sort_values(by='Count', ascending=False)

    # Save to CSV
    df_results.to_csv(output_filename, index=False)
    print(f'Results saved to {output_filename}')

# Main function to run the process
def main():
    url = 'https://api.lotterywest.wa.gov.au/api/v1/games/5130/results-csv?_gl=1*okyk8r*_gcl_aw*R0NMLjE3MjkyNDUyNDguQ2p3S0NBandqc2k0QmhCNUVpd0FGQUwwWU12ZzVBZU9SbUxGcjkzNTFhZ0lHcl9jcW1wM3ZUX3dxUkpSYlU3Um5KZl9VdDNiV3M1bllSb0NxdnNRQXZEX0J3RQ..*_gcl_dc*R0NMLjE3MjkyNDUyNDguQ2p3S0NBandqc2k0QmhCNUVpd0FGQUwwWU12ZzVBZU9SbUxGcjkzNTFhZ0lHcl9jcW1wM3ZUX3dxUkpSYlU3Um5KZl9VdDNiV3M1bllSb0NxdnNRQXZEX0J3RQ..*_gcl_au*MTg3Mzc2Njg4Ny4xNzI5MjQ1MjEw*_ga*MTI5Nzg4ODg0LjE3MjkyNDUyMTE.*_ga_KTQ5JJBDHN*MTcyOTI0NTIxMC4xLjEuMTcyOTI0NTI0Ny4yMy4wLjA.'  # Lotto results URL
    csv_data = download_lotto_results(url)

    if csv_data:
        ibis_table = load_lotto_results(csv_data)
        combination_counts = count_combinations_with_dates(ibis_table)
        
        # Save results to CSV
        output_filename = 'csv/ozlotto_combo_results.csv'
        save_counts_to_csv(combination_counts, output_filename)

if __name__ == '__main__':
    main()


Results saved to ozlotto_combo_results.csv
